# Euclidean distance, step by step

In [1]:
import numpy as np

# Step 1: Create two random arrays of 5 elements each
array1 = np.random.rand(5)
array2 = np.random.rand(5)
# Are oposite
#array1 = np.zeros(5)
#array2 = np.ones(5)
# Are equal
#array1 = np.ones(5)
#array2 = np.ones(5)

# Step 2: Display the arrays
print("Array 1:", array1)
print("Array 2:", array2)

Array 1: [0.82687314 0.52844577 0.64078148 0.39368252 0.17107901]
Array 2: [0.07946117 0.09498804 0.89215013 0.28948793 0.43523078]


In [2]:
# Step 3: Calculate the difference between corresponding elements
difference = array1 - array2
print("Difference:", difference)

Difference: [ 0.74741197  0.43345773 -0.25136866  0.10419458 -0.26415177]


In [3]:
# Step 4: Square the differences
squared_difference = difference**2
print("Squared Difference:", squared_difference)

Squared Difference: [0.55862466 0.1878856  0.0631862  0.01085651 0.06977616]


In [4]:
# Step 5: Sum the squared differences
sum_squared_difference = np.sum(squared_difference)
print("Sum of Squared Differences:", sum_squared_difference)

Sum of Squared Differences: 0.8903291273884621


In [5]:
# Step 6: Take the square root of the sum
euclidean_distance = np.sqrt(sum_squared_difference)
print("Euclidean Distance:", euclidean_distance)

Euclidean Distance: 0.9435725342486725


# GIL (Python Global Interpreter Lock)

- Lock that allows only one thread to hold the control of the Python interpreter.
- This means that only one thread can be in a state of execution at any point in time. 
- The impact of the GIL isn’t visible to developers who execute single-threaded programs, but it can be a performance bottleneck in CPU-bound and multi-threaded code

## Why was the GIL implemented in Python?

- Python uses reference counting for memory management. It means that objects created in Python have a reference count variable that keeps track of the number of references that point to the object. 
- When this count reaches zero, the memory occupied by the object is released.

In [9]:
import sys
a = []
b = a
sys.getrefcount(a)

3

In [7]:
import sys
a = []
b = a
c = a
sys.getrefcount(a)

4

The problem was that this reference count variable needed protection from race conditions where two threads increase or decrease its value simultaneously. This can cause the following:
- It can cause either leaked memory that is never released.
- Release the memory while a reference to that object still exists. 

This reference count variable can be kept safe by adding locks to all data structures that are shared across threads so that they are not modified inconsistently.

But adding a lock to each object or groups of objects means multiple locks will exist which can cause another problem—Deadlocks (deadlocks can only happen if there is more than one lock) and can also decrease performance.

The GIL is a single lock on the interpreter itself which adds a rule that execution of any Python bytecode requires acquiring the interpreter lock. This prevents deadlocks (as there is only one lock) and doesn’t introduce much performance overhead. **But it effectively makes any CPU-bound Python program single-threaded.**

## Why Python still has the GIL?

Python cannot bring a change as significant as the removal of GIL without causing backward incompatibility issues.

Discussed by the creator of Python:

https://www.artima.com/weblogs/viewpost.jsp?thread=214235


**Alternative Python interpreters:** Python has multiple interpreter implementations. CPython, Jython, IronPython and PyPy, written in C, Java, C# and Python respectively, are the most popular ones. GIL exists only in the original Python implementation that is CPython. If your program, with its libraries, is available for one of the other implementations then you can try them out as well.

In [11]:
from numba import jit
import numpy as np
import time

x = np.arange(100).reshape(10, 10)

@jit(nopython=True)
def go_fast(a): # Function is compiled and runs in machine code
    trace = 0.0
    for i in range(a.shape[0]):
        trace += np.tanh(a[i, i])
    return a + trace

# COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
go_fast(x)
end = time.time()
print("Elapsed (with compilation) = {}s".format((end - start)))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
go_fast(x)
end = time.time()
print("Elapsed (after compilation) = {}s".format((end - start)))

Elapsed (with compilation) = 0.8089010715484619s
Elapsed (after compilation) = 5.888938903808594e-05s
